In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [43]:
clean_9000 = 'result_clean_pdf_9000_files.csv'
clean_109 = 'result_clean_pdf_109_embed_video.csv'
malw_173 = 'result_malware_pdf_cve_sorted_173_files.csv'
malw_10982 = 'result_malware_pdf_pre_04-2011_10982_files.csv'
df_clean_9000 = pd.read_csv(clean_9000, sep=',')
df_clean_109 = pd.read_csv(clean_109, sep=',')
df_malw_173 = pd.read_csv(malw_173, sep=',')
df_malw_10982 = pd.read_csv(malw_10982, sep=',')

In [44]:
df_clean = df_clean_9000.append(df_clean_109, ignore_index=True)
df_malw = df_malw_10982.append(df_malw_173, ignore_index=True)

/var/folders/48/1_svc68j0_b2js686ls_zqhh0000gn/T/ipykernel_43727/1789017819.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_clean = df_clean_9000.append(df_clean_109, ignore_index=True)
/var/folders/48/1_svc68j0_b2js686ls_zqhh0000gn/T/ipykernel_43727/1789017819.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_malw = df_malw_10982.append(df_malw_173, ignore_index=True)


In [67]:
df_clean["malware"] = np.zeros(df_clean.shape[0], dtype='bool')
df_malw["malware"] = np.ones(df_malw.shape[0], dtype='bool')
df_tot = df_clean.append(df_malw, ignore_index=True)
df_train, df_test = train_test_split(df_tot, test_size=0.1)

/var/folders/48/1_svc68j0_b2js686ls_zqhh0000gn/T/ipykernel_43727/2443839953.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tot = df_clean.append(df_malw, ignore_index=True)


In [143]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(df_train.iloc[:,:-1], df_train.iloc[:,-1:])
y_hat = knn.predict(df_test.iloc[:,:-1])
y = df_test.iloc[:,-1:].values.ravel()
#print(y_hat.sum() / df_test.shape[0]==True)
#for i in range(y_ha)
pred = y == y_hat
pred.sum() / len(y)

/Users/mattiarosso/opt/anaconda3/envs/malis/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.9990103908955963